In [34]:

USE ERP
GO
CREATE PROCEDURE usp_DropDatabase
    @DatabaseName NVARCHAR(128)
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @KillCommand NVARCHAR(MAX);
    DECLARE @Sql NVARCHAR(MAX);

    -- Initialize the command to kill active connections
    SET @KillCommand = '';

    -- Build the kill command for each active connection, excluding the current session
    SELECT @KillCommand = @KillCommand + 'KILL ' + CONVERT(NVARCHAR(10), session_id) + ';'
    FROM sys.dm_exec_sessions
    WHERE database_id = DB_ID(@DatabaseName) AND session_id <> @@SPID;

    -- Execute the kill commands
    EXEC sp_executesql @KillCommand;

    -- Drop the database
    SET @Sql = N'DROP DATABASE IF EXISTS [' + @DatabaseName + ']';
    EXEC sp_executesql @Sql;
END

GO

/*
Anlegen der grundsätzlichen Datenbankobjekte (Datenbank, Schema)
*/

-- Ausführen der Helper Stored Procedure um die Datenbank zu löschen
EXEC usp_DropDatabase 'dwh'

CREATE DATABASE dwh COLLATE SQL_Latin1_General_CP1_CI_AS; -- Gleiche Collation wie die ERP Datenbank


Commands completed successfully.

: Msg 2714, Level 16, State 3, Procedure usp_DropDatabase, Line 1
There is already an object named 'usp_DropDatabase' in the database.

Commands completed successfully.

Total execution time: 00:00:01.145

In [35]:
USE DWH;
GO
CREATE PROCEDURE Transfer_TopSeller
(
    @SourceDB VARCHAR(50) = 'ERP',
    @SourceTable1 VARCHAR(50)= 'Sales.SalesOrderHeader',
    @DestinationTable1 VARCHAR(50)= 'Staging_TopSeller.SalesOrderHeader',
    @SourceTable2 VARCHAR(50) = 'Sales.SalesOrderDetail',
    @DestinationTable2 VARCHAR(50) = 'Staging_TopSeller.SalesOrderDetail',
    @SourceTable3 VARCHAR(50) = 'Sales.SalesTerritory',
    @DestinationTable3 VARCHAR(50) = 'Staging_TopSeller.SalesTerritory',
    @DestinationDB VARCHAR(50) = 'DWH'
)
AS
BEGIN
    DECLARE @SQL NVARCHAR(MAX);
    DECLARE @FullyQualifiedDestinationTable NVARCHAR(128);

    BEGIN TRY
        -- Erste Tabelle kopieren
        SET @FullyQualifiedDestinationTable = @DestinationDB + '.' + @DestinationTable1;
        IF OBJECT_ID(@FullyQualifiedDestinationTable, 'U') IS NOT NULL
        BEGIN
            SET @SQL = 'DROP TABLE ' + @FullyQualifiedDestinationTable;
            EXEC sp_executesql @SQL;
        END
        SET @SQL = 'SELECT * INTO ' + @FullyQualifiedDestinationTable + ' FROM ' + @SourceDB + '.' + @SourceTable1;
        EXEC sp_executesql @SQL;

        -- Zweite Tabelle kopieren
        SET @FullyQualifiedDestinationTable = @DestinationDB + '.' + @DestinationTable2;
        IF OBJECT_ID(@FullyQualifiedDestinationTable, 'U') IS NOT NULL
        BEGIN
            SET @SQL = 'DROP TABLE ' + @FullyQualifiedDestinationTable;
            EXEC sp_executesql @SQL;
        END
        SET @SQL = 'SELECT * INTO ' + @FullyQualifiedDestinationTable + ' FROM ' + @SourceDB + '.' + @SourceTable2;
        EXEC sp_executesql @SQL;

        -- Dritte Tabelle kopieren
        SET @FullyQualifiedDestinationTable = @DestinationDB + '.' + @DestinationTable3;
        IF OBJECT_ID(@FullyQualifiedDestinationTable, 'U') IS NOT NULL
        BEGIN
            SET @SQL = 'DROP TABLE ' + @FullyQualifiedDestinationTable;
            EXEC sp_executesql @SQL;
        END
        SET @SQL = 'SELECT * INTO ' + @FullyQualifiedDestinationTable + ' FROM ' + @SourceDB + '.' + @SourceTable3;
        EXEC sp_executesql @SQL;
    END TRY
    BEGIN CATCH
        PRINT 'Ein Fehler ist aufgetreten: ' + ERROR_MESSAGE();
        THROW;
    END CATCH
END;
GO


Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.140

In [36]:
USE DWH;
GO
CREATE PROCEDURE Transfer_SalesGrowth
(
    @SourceDB VARCHAR(50) = 'ERP',
    @SourceTable1 VARCHAR(50) = 'Sales.SalesOrderHeader',
    @DestinationTable1 VARCHAR(50) = 'Staging_SalesGrowth.SalesOrderHeader',
    @SourceTable2 VARCHAR(50) = 'Sales.SalesOrderDetail',
    @DestinationTable2 VARCHAR(50) = 'Staging_SalesGrowth.SalesOrderDetail',
    @SourceTable3 VARCHAR(50) = 'Production.Product',
    @DestinationTable3 VARCHAR(50) = 'Staging_SalesGrowth.Product',
    @DestinationDB VARCHAR(50) = 'DWH'
)
AS
BEGIN
    DECLARE @SQL NVARCHAR(MAX);
    DECLARE @FullyQualifiedDestinationTable NVARCHAR(128);

    BEGIN TRY
        -- Erste Tabelle kopieren
        SET @FullyQualifiedDestinationTable = @DestinationDB + '.' + @DestinationTable1;
        IF OBJECT_ID(@FullyQualifiedDestinationTable, 'U') IS NOT NULL
        BEGIN
            SET @SQL = 'DROP TABLE ' + @FullyQualifiedDestinationTable;
            EXEC sp_executesql @SQL;
        END
        SET @SQL = 'SELECT * INTO ' + @FullyQualifiedDestinationTable + ' FROM ' + @SourceDB + '.' + @SourceTable1;
        EXEC sp_executesql @SQL;

        -- Zweite Tabelle kopieren
        SET @FullyQualifiedDestinationTable = @DestinationDB + '.' + @DestinationTable2;
        IF OBJECT_ID(@FullyQualifiedDestinationTable, 'U') IS NOT NULL
        BEGIN
            SET @SQL = 'DROP TABLE ' + @FullyQualifiedDestinationTable;
            EXEC sp_executesql @SQL;
        END
        SET @SQL = 'SELECT * INTO ' + @FullyQualifiedDestinationTable + ' FROM ' + @SourceDB + '.' + @SourceTable2;
        EXEC sp_executesql @SQL;

        -- Dritte Tabelle kopieren
        SET @FullyQualifiedDestinationTable = @DestinationDB + '.' + @DestinationTable3;
        IF OBJECT_ID(@FullyQualifiedDestinationTable, 'U') IS NOT NULL
        BEGIN
            SET @SQL = 'DROP TABLE ' + @FullyQualifiedDestinationTable;
            EXEC sp_executesql @SQL;
        END
        SET @SQL = 'SELECT * INTO ' + @FullyQualifiedDestinationTable + ' FROM ' + @SourceDB + '.' + @SourceTable3;
        EXEC sp_executesql @SQL;
    END TRY
    BEGIN CATCH
        PRINT 'Ein Fehler ist aufgetreten: ' + ERROR_MESSAGE();
        THROW;
    END CATCH
END;


Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.131

In [37]:
USE DWH;
GO

CREATE PROCEDURE ETL_TopSeller_ProductID
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @TableToCreate NVARCHAR(128) = 'DWH.DWH_SalesGrowth.Dim_ProductID';
    DECLARE @SQL NVARCHAR(MAX);

    BEGIN TRY

    IF OBJECT_ID(@TableToCreate, 'U') IS NOT NULL
    BEGIN
        SET @SQL = 'DROP TABLE ' + @TableToCreate;
        EXEC sp_executesql @SQL;
    END

    SET @SQL = 'CREATE TABLE ' + @TableToCreate + ' (
        ProductID INT PRIMARY KEY,
        Produktname VARCHAR(MAX)
    )';
    EXEC sp_executesql @SQL;

-- Define the SELECT DISTINCT AND THE COLUMNS we want to select
    SET @SQL = 'INSERT INTO ' + @TableToCreate + ' (ProductID, Produktname)
    SELECT DISTINCT pr.ProductID, 
pr.Name as Produktname
FROM  
    Staging_SalesGrowth.SalesOrderHeader sh  
LEFT OUTER JOIN  
    Staging_SalesGrowth.SalesOrderDetail sd ON sh.SalesOrderID = sd.SalesOrderID 
LEFT OUTER JOIN  
    Staging_SalesGrowth.Product pr ON sd.ProductID = pr.ProductID';
    EXEC sp_executesql @SQL;

    END TRY
    BEGIN CATCH
        PRINT 'Ein Fehler ist aufgetreten: ' + ERROR_MESSAGE();
        THROW;
    END CATCH
END;
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.117

In [ ]:
USE DWH;
GO

CREATE PROCEDURE "ETL_TerritorySales_TerritoryID"
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @TableToCreate NVARCHAR(128) = 'DWH.DWH_TopSeller.Dim_TerritoryID';
    DECLARE @SQL NVARCHAR(MAX);

    BEGIN TRY

    IF OBJECT_ID(@TableToCreate, 'U') IS NOT NULL
    BEGIN
        SET @SQL = 'DROP TABLE ' + @TableToCreate;
        EXEC sp_executesql @SQL;
    END

    SET @SQL = 'CREATE TABLE ' + @TableToCreate + ' (
        territoryID INT PRIMARY KEY,
        Territory VARCHAR(MAX)
    )';
    EXEC sp_executesql @SQL;

    SET @SQL = 'INSERT INTO ' + @TableToCreate + ' (territoryID, Territory)
    SELECT DISTINCT sap.territoryID, 
st.Name as Territory
from Staging_TopSeller.SalesPerson sap 
left outer join Staging_TopSeller.SalesTerritory st on st.territoryID = sap.TerritoryID
WHERE sap.territoryID IS NOT NULL';
    EXEC sp_executesql @SQL;
    END TRY
    BEGIN CATCH
        PRINT 'Ein Fehler ist aufgetreten: ' + ERROR_MESSAGE();
        THROW;
    END CATCH
END;
GO

In [38]:
CREATE SCHEMA Staging_TopSeller

Commands completed successfully.

Total execution time: 00:00:00.051

In [39]:
CREATE SCHEMA Staging_SalesGrowth

Commands completed successfully.

Total execution time: 00:00:00.084

In [40]:
CREATE SCHEMA DWH_SalesGrowth

Commands completed successfully.

Total execution time: 00:00:00.055

In [41]:
CREATE SCHEMA DWH_TopSeller

Commands completed successfully.

Total execution time: 00:00:00.050

In [42]:
EXEC Transfer_TopSeller;
EXEC Transfer_SalesGrowth
EXEC ETL_TopSeller_ProductID

(31465 rows affected)

(121317 rows affected)

(31465 rows affected)

(121317 rows affected)

(504 rows affected)

Total execution time: 00:00:02.021